In [35]:
import requests
import json
import os

In [36]:
ICC_SCHEDULE_URL = "https://assets-icc.sportz.io/cricket/v1/schedule?client_id=tPZJbRgIub3Vua93%2FDWtyQ%3D%3D&feed_format=json&from_date=20000101&competition_type_ids=1&lang=en&league_ids=1%2C9&pagination=false&timezone=0530&to_date=20260101&timezone=0530"

response = requests.get(ICC_SCHEDULE_URL, timeout=100)
data = response.json()


os.makedirs("icc", exist_ok=True)

with open("icc/tests_2000_2025.json", 'w') as f:
    json.dump(data, f, indent=4)

In [37]:
# ICC_COMMS_URL = "https://assets-icc.sportz.io/cricket/static/json/iccteamlist/womens_international_teamlist.json"

# response = requests.get(ICC_COMMS_URL, timeout=100)
# data = response.json()

# with open("icc/icc_womens_international_teamlist.json", "w") as f:
#     json.dump(data, f, indent=4)

In [38]:
# ICC_COMMS_URL = "https://assets-icc.sportz.io/cricket/v1/game/commentary?client_id=tPZJbRgIub3Vua93/DWtyQ==&game_id=187175&pagination=false&inning=1&type=all&lang=en&feed_format=json"
# ICC_PLAYER_URL = "https://assets-icc.sportz.io/cricket/v1/player?client_id=tPZJbRgIub3Vua93/DWtyQ==&feed_format=json&player_id=74837&lang=en"
# ICC_SCORECARD_URL = "https://assets-icc.sportz.io/cricket/v1/game/scorecard?client_id=tPZJbRgIub3Vua93/DWtyQ==&game_id=1&lang=en&feed_format=json"

# response = requests.get(ICC_COMMS_URL, timeout=100)
# data = response.json()

# data

In [39]:
# import requests
# import json

# def fetch_free_proxies():
#     """Fetch free proxies from a reliable source"""
#     try:
#         # Free proxy list
#         response = requests.get('https://free-proxy-list.net/')
#         proxies = []
        
#         # Parse the HTML (you'll need to install beautifulsoup4)
#         from bs4 import BeautifulSoup
#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         for row in soup.find('table', {'class': 'table table-striped table-bordered'}).find_all('tr')[1:]:
#             cols = row.find_all('td')
#             if len(cols) > 1:
#                 ip = cols[0].text.strip()
#                 port = cols[1].text.strip()
#                 proxies.append(f'http://{ip}:{port}')
        
#         return proxies[:10]  # Return first 10 proxies
#     except Exception as e:
#         print(f"Error fetching proxies: {e}")
#         return []

# def test_proxy(proxy, url='http://httpbin.org/ip'):
#     """Test if a single proxy works"""
#     try:
#         response = requests.get(
#             url, 
#             proxies={'http': proxy, 'https': proxy}, 
#             timeout=5
#         )
#         return response.status_code == 200
#     except:
#         return False

# def get_working_proxies(proxies, max_tries=5):
#     """Filter and return working proxies"""
#     working_proxies = []
#     for proxy in proxies:
#         if test_proxy(proxy):
#             working_proxies.append(proxy)
#         if len(working_proxies) == max_tries:
#             break
#     return working_proxies

# # Main script
# def make_request_with_proxy(url):
#     # Fetch proxies
#     proxies = fetch_free_proxies()
    
#     # Get working proxies
#     working_proxies = get_working_proxies(proxies)
    
#     if not working_proxies:
#         print("No working proxies found!")
#         return None
    
#     # Try each working proxy
#     for proxy in working_proxies:
#         try:
#             response = requests.get(
#                 url, 
#                 proxies={'http': proxy, 'https': proxy}, 
#                 timeout=10
#             )
#             response.raise_for_status()
#             return response
#         except Exception as e:
#             print(f"Proxy {proxy} failed: {e}")
    
#     return None

# # Example usage
# url = "https://assets-icc.sportz.io/cricket/v1/schedule?client_id=tPZJbRgIub3Vua93%2FDWtyQ%3D%3D&feed_format=json&from_date=18000101&competition_type_ids=1&lang=en&league_ids=1%2C9&pagination=false&timezone=0530&to_date=19000101&timezone=0530"
# response = make_request_with_proxy(url)
# data = response.json()

# if response:
#     print("Request successful!")
#     with open("icc/tests_1800_1900.json", 'w') as f:
#         json.dump(data, f, indent=4)
# else:
#     print("Could not make request with any proxy")

In [40]:
import pandas as pd

df_18_19 = pd.read_json('./icc/tests_1800_1900.json')
df_19_20 = pd.read_json('./icc/tests_1900_2000.json')
df_20_21 = pd.read_json('./icc/tests_2000_2025.json')

match_df_18_19 = pd.DataFrame(df_18_19['data']['matches'])
match_df_19_20 = pd.DataFrame(df_19_20['data']['matches'])
match_df_20_21 = pd.DataFrame(df_20_21['data']['matches'])

In [41]:
columns_only_in_df1 = set(match_df_20_21.columns) - set(match_df_19_20.columns)
columns_only_in_df1

{'day',
 'is_provisional_date',
 'is_provisional_series_date',
 'is_provisional_time',
 'session'}

In [42]:
all_tests_df = pd.concat([match_df_18_19, match_df_19_20, match_df_20_21], axis=0, ignore_index=True)

In [43]:
def reshape_scores(scores):
    single_row = {}
    for inning in scores:
        inning_no = inning['inning_no']
        for key, value in inning.items():
            if key != 'inning_no':  # Exclude 'inning_no' from the column names
                column_name = f"inning_{inning_no}_{key}"
                single_row[column_name] = value
    return single_row

In [44]:
reshaped_data = all_tests_df['scores'].apply(reshape_scores)
reshaped_df = pd.DataFrame(reshaped_data.tolist())
all_tests_df = pd.concat([all_tests_df.drop(columns='scores'), reshaped_df], axis=1)

# all_tests_df = pd.concat([all_tests_df.drop("award", axis=1), all_tests_df["award"].apply(pd.Series).add_prefix('award')], axis=1)

In [45]:
columns_only_in_df1 = set(all_tests_df.columns) - set(match_df_20_21.columns)
# columns_only_in_df1

In [46]:
def reshape_other_info(info):
    # Flatten the dictionary into key-value pairs
    return info

reshaped_other_info = all_tests_df['other_info'].apply(reshape_other_info)
reshaped_other_df = pd.DataFrame(reshaped_other_info.tolist())
all_tests_df = pd.concat([all_tests_df.drop(columns='other_info'), reshaped_other_df], axis=1)

In [47]:
all_tests_df.columns

Index(['is_deleted', 'start_date', 'day_night', 'gmt_offset', 'group',
       'league', 'league_id', 'live', 'live_coverage', 'comp_type_id',
       ...
       'inning_1_is_declare', 'inning_2_is_declare', 'inning_4_target',
       'inning_2_is_followon', 'match_center', 'watch_live', 'high_light',
       'blacklisted_countries', 'whitelisted_countries', 'ticket_link'],
      dtype='object', length=136)

In [48]:
all_tests_df.to_csv("icc/icc_tests_info.csv", index=False)

In [49]:
icc_match_id_df = all_tests_df['match_id']
icc_match_id_df.to_csv("icc/icc_tests_match_id.txt", index=False)

In [50]:
latest_icc_match_id_df = all_tests_df['match_id'][all_tests_df['match_status'] == 'Stumps']
latest_icc_match_id_df.to_csv("icc/icc_tests_comms_match_id.txt", index=False)
latest_icc_match_id_df

Series([], Name: match_id, dtype: object)

In [51]:
import requests
import json

def fetch_free_proxies():
    """Fetch free proxies from a reliable source"""
    try:
        # Free proxy list
        response = requests.get('https://free-proxy-list.net/')
        proxies = []
        
        # Parse the HTML (you'll need to install beautifulsoup4)
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for row in soup.find('table', {'class': 'table table-striped table-bordered'}).find_all('tr')[1:]:
            cols = row.find_all('td')
            if len(cols) > 1:
                ip = cols[0].text.strip()
                port = cols[1].text.strip()
                proxies.append(f'http://{ip}:{port}')
        
        return proxies[:10]  # Return first 10 proxies
    except Exception as e:
        print(f"Error fetching proxies: {e}")
        return []

def test_proxy(proxy, url='http://httpbin.org/ip'):
    """Test if a single proxy works"""
    try:
        response = requests.get(
            url, 
            proxies={'http': proxy, 'https': proxy}, 
            timeout=5
        )
        return response.status_code == 200
    except:
        return False

def get_working_proxies(proxies, max_tries=5):
    """Filter and return working proxies"""
    working_proxies = []
    for proxy in proxies:
        if test_proxy(proxy):
            working_proxies.append(proxy)
        if len(working_proxies) == max_tries:
            break
    return working_proxies

# Main script
def make_request_with_proxy(url):
    # Fetch proxies
    proxies = fetch_free_proxies()
    
    # Get working proxies
    working_proxies = get_working_proxies(proxies)
    
    if not working_proxies:
        print("No working proxies found!")
        return None
    
    # Try each working proxy
    for proxy in working_proxies:
        try:
            response = requests.get(
                url, 
                proxies={'http': proxy, 'https': proxy}, 
                timeout=10
            )
            response.raise_for_status()
            return response
        except Exception as e:
            print(f"Proxy {proxy} failed: {e}")
    
    return None

# Example usage
# url = "https://assets-icc.sportz.io/cricket/v1/schedule?client_id=tPZJbRgIub3Vua93%2FDWtyQ%3D%3D&feed_format=json&from_date=18000101&competition_type_ids=1&lang=en&league_ids=1%2C9&pagination=false&timezone=0530&to_date=19000101&timezone=0530"
# response = make_request_with_proxy(url)
# data = response.json()

In [52]:
def icc_comms_info(match_id):

    match_comms_data = []

    for inn_no in range(1, 5):

        ICC_COMMS_URL = f"https://assets-icc.sportz.io/cricket/v1/game/commentary?client_id=tPZJbRgIub3Vua93/DWtyQ==&game_id={match_id}&pagination=false&inning={inn_no}&type=all&lang=en&feed_format=json"
        response = requests.get(ICC_COMMS_URL, timeout=100)
        # response = make_request_with_proxy(ICC_COMMS_URL)
        data = response.json()

        if response:
            print("Request successful!")
        else:
            print("Could not make request with any proxy")
        
        if(data['data'] != None):
            print(f"{match_id} INN {inn_no} MATCH COMMS THERE!!!", match_id)
            match_comms_data.append(data)

    with open(f"icc/test_matches/{match_id}.json", 'w') as f:
        json.dump(match_comms_data, f, indent=4)
    # if(match_comms_data == []):
    #     print(f"{match_df} NO COMMS!!", match_id)
    # else:
    #     print(f"{match_id} DONE!!!", match_id)
        
    # return match_comms_data

In [53]:

match_ids = []

with open('icc/icc_tests_comms_match_id.txt', 'r') as file:
    for l_no, i in enumerate(file):
        if(l_no != 0):
            match_ids.append(i.strip())
    

print(len(match_ids))
for match_no in match_ids:
    icc_comms_info(match_no)

0


In [54]:
# def reshape_comms_info(info):
#     # Flatten the dictionary into key-value pairs
#     return info

In [55]:
# def reshape_comms_info(info):
#     # Check if the info is a dictionary
#     if isinstance(info, dict):
#         # Flatten the dictionary into key-value pairs
#         return info
#     elif isinstance(info, list):
#         # If the info is a list, create a dictionary with indexes
#         return {f"item_{i}": item for i, item in enumerate(info)}
#     else:
#         # If it's neither a dictionary nor a list, return it as a single value
#         return {0: info}


In [56]:
import pandas as pd

def icc_comms_cleaner(match_id):

    df = pd.read_json(f"./icc/test_matches/{match_id}.json")
    final_test_df = pd.DataFrame()

    for i in range(len(df)):
        test_df = pd.DataFrame(df['data'][i]['Commentary'])
        test_df = test_df.iloc[::-1].reset_index(drop=True)
        test_df = test_df[test_df['Isball'] == True]

        inn_no = i + 1
        test_df.insert(0, 'Innings', inn_no)
        test_df.insert(0, 'MatchID', match_id)

        # if 'Batsman_Details' in test_df.columns:

        #     reshaped_bat_det_info = test_df['Batsman_Details'].apply(reshape_comms_info)
        #     reshaped_non_stkr_det_info = test_df['Non_Striker_Details'].apply(reshape_comms_info)
        #     reshaped_bow_det_info = test_df['Bowler_Details'].apply(reshape_comms_info)

        #     # Convert the reshaped data into a DataFrame
        #     reshaped_bat_det_info = pd.DataFrame(reshaped_bat_det_info.tolist())
        #     reshaped_non_stkr_det_info = pd.DataFrame(reshaped_non_stkr_det_info.tolist())
        #     reshaped_bow_det_info = pd.DataFrame(reshaped_bow_det_info.tolist())

        #     reshaped_bat_det_info.columns = [f"curr_striker_{col}" for col in reshaped_bat_det_info.columns]
        #     reshaped_non_stkr_det_info.columns = [f"curr_non_striker_{col}" for col in reshaped_non_stkr_det_info.columns]
        #     reshaped_bow_det_info.columns = [f"curr_bowler_{col}" for col in reshaped_bow_det_info.columns]

        #     # Get the original index of the 'other_info' column

        #     # non_stkr_index = test_df.columns.get_loc('Non_Striker_Details')
        #     # bowler_index = test_df.columns.get_loc('Bowler_Details')
        #     # bat_index = test_df.columns.get_loc('Batsman_Details')

        #     # Drop the 'other_info' column

        #     test_df = pd.concat([test_df, reshaped_non_stkr_det_info, reshaped_bow_det_info, reshaped_bat_det_info], axis=1)
            
        #     test_df = test_df.drop(columns=['Non_Striker_Details', 'Bowler_Details', 'Batsman_Details'])

        #     # Insert reshaped data with new column names where 'other_info' was
        #     for column_name in reshaped_non_stkr_det_info.columns:
        #         test_df.insert(non_stkr_index, column_name, reshaped_non_stkr_det_info[column_name])
        #         non_stkr_index += 1

        #     for column_name in reshaped_bow_det_info.columns:
        #         test_df.insert(bowler_index, column_name, reshaped_bow_det_info[column_name])
        #         bowler_index += 1

        #     for column_name in reshaped_bat_det_info.columns:
        #         test_df.insert(bat_index, column_name, reshaped_bat_det_info[column_name])
        #         bat_index += 1
            
            # test_df.drop(columns=['UID', 'OID', 'Bowler_Short_Name', 'Batsman_Short_Name', 'Non_Striker_Short_Name', 'This_Over', 'Ball_Speed_This_Over', 'Default_Commentary', 'Commentary'], inplace=True)
            
        # if 'Summary' in test_df.columns:
            
        #     if 'curr_striker_Batsman' in test_df.columns:
        #         break
        #     else:
        #         reshaped_sum_info = test_df['Summary'].apply(reshape_comms_info)
        #         reshaped_sum_info = pd.DataFrame(reshaped_sum_info.tolist())
        #         reshaped_sum_info.columns = [f"summary_{col}" for col in reshaped_sum_info.columns]
        #         test_df = test_df.drop(columns=['Summary'])
        #         test_df = pd.concat([test_df, reshaped_sum_info], axis=1)


        final_test_df = pd.concat([final_test_df, test_df], axis=0, ignore_index=True)


    final_test_df.to_csv(f"./icc/test_matches/{match_id}.csv")
    

In [57]:
for match_no in match_ids:
    icc_comms_cleaner(match_no)

In [58]:
import pandas as pd
import glob

csv_dir = "./icc/test_matches/"
csv_files = glob.glob(csv_dir + "*.csv")
dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    # dfs.append(df)

    if 'Timestamp' in df.columns and not df['Timestamp'].empty:
        # Check if the first value in the 'Timestamp' column is valid
        try:
            first_timestamp = pd.to_datetime(df['Timestamp'].iloc[0])  # Get the first timestamp and convert to datetime
            dfs.append((df, first_timestamp))
        except Exception as e:
            print(f"Error parsing timestamp in {file}: {e}")
    else:
        print(f"Skipping file {file} due to missing or empty 'Timestamp' column")

dfs.sort(key=lambda x: x[1])
sorted_dfs = [df for df, _ in dfs]

final_df = pd.concat(sorted_dfs, axis=0 ,ignore_index=True)
final_df.to_csv("./icc/icc_tests_bbb.csv", index=False)

Skipping file ./icc/test_matches/253604.csv due to missing or empty 'Timestamp' column
Skipping file ./icc/test_matches/243095.csv due to missing or empty 'Timestamp' column
Skipping file ./icc/test_matches/244852.csv due to missing or empty 'Timestamp' column
Skipping file ./icc/test_matches/249225.csv due to missing or empty 'Timestamp' column


In [59]:
import pandas as pd

icc_tests_bbb_df = pd.read_csv("./icc/icc_tests_bbb.csv")

/tmp/ipykernel_25545/2400456527.py:3: DtypeWarning: Columns (19,21,22,25,26,28,29,30,34,37,38,39,40,41,44,45,46,47,50,51,52,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  icc_tests_bbb_df = pd.read_csv("./icc/icc_tests_bbb.csv")


In [60]:
icc_tests_bbb_df.columns

Index(['Unnamed: 0', 'MatchID', 'Innings', 'Over', 'Id', 'Timestamp', 'Runs',
       'ZAD', 'Detail', 'Isball', 'Bowler', 'Bowler_Name', 'Batsman',
       'Batsman_Name', 'Commentary', 'Iswicket', 'Dismissed', 'Isboundary',
       'Summary', 'Batsman_Style', 'Non_Striker', 'Non_Striker_Name', 'Score',
       'Batsman_Runs', 'Bowler_Conceded_Runs', 'Dismissal_Type',
       'Dismissal_Id', 'Extras_Runs', 'Ball_Speed', 'Howout', 'Fielders',
       'Ball_Number', 'Day', 'Session', 'Milestone', 'Over_No', 'Ball',
       'Ball_Line_Length', 'This_Over', 'Batsman_Details',
       'Non_Striker_Details', 'Bowler_Details', 'Is_Nonball_Wicket',
       'Edited_Commentary', 'End_Over', 'Default_Commentary', 'Review_Detail',
       'Run_Out_Miss', 'UID', 'OID', 'Bowler_Short_Name', 'Batsman_Short_Name',
       'Non_Striker_Short_Name', 'Ball_Speed_This_Over', 'Created_Timestamp',
       'IsFreehit'],
      dtype='object')

In [61]:
import pandas as pd
import numpy as np

def reshape_comms_info(details):
    """
    Reshape the Batsman, Non-Striker, or Bowler details.
    Handles NaN and string parsing for dictionary-like structures.
    """
    # If details is NaN or None, return an empty dictionary
    if pd.isna(details) or details is None:
        return {}

    # If details is a string (e.g., a JSON-like string), try to parse it
    if isinstance(details, str):
        try:
            # Safe evaluation of the string to a dictionary
            details = eval(details)
        except:
            # Return an empty dictionary if parsing fails
            return {}

    # If the details are a dictionary, process it
    if isinstance(details, dict):
        reshaped_data = {}
        for key, value in details.items():
            reshaped_data[key] = value
        return reshaped_data

    # If the details are neither NaN nor a string or dictionary, return an empty dictionary
    return {}

In [62]:
reshaped_bat_det_info = icc_tests_bbb_df['Batsman_Details'].apply(reshape_comms_info)
reshaped_non_stkr_det_info = icc_tests_bbb_df['Non_Striker_Details'].apply(reshape_comms_info)
reshaped_bow_det_info = icc_tests_bbb_df['Bowler_Details'].apply(reshape_comms_info)

# Convert the reshaped lists into DataFrames
reshaped_bat_det_info_df = pd.DataFrame(reshaped_bat_det_info.tolist())
reshaped_non_stkr_det_info_df = pd.DataFrame(reshaped_non_stkr_det_info.tolist())
reshaped_bow_det_info_df = pd.DataFrame(reshaped_bow_det_info.tolist())

# Add suffixes based on the role for each of the DataFrames
reshaped_bat_det_info_df.columns = [f"curr_striker_{col}" for col in reshaped_bat_det_info_df.columns]
reshaped_non_stkr_det_info_df.columns = [f"curr_non_striker_{col}" for col in reshaped_non_stkr_det_info_df.columns]
reshaped_bow_det_info_df.columns = [f"curr_bowler_{col}" for col in reshaped_bow_det_info_df.columns]

# Concatenate the reshaped DataFrames back into the original DataFrame
icc_tests_bbb_df = pd.concat([icc_tests_bbb_df, reshaped_non_stkr_det_info_df, reshaped_bow_det_info_df, reshaped_bat_det_info_df], axis=1)

# Drop the original details columns
icc_tests_bbb_df = icc_tests_bbb_df.drop(columns=['Non_Striker_Details', 'Bowler_Details', 'Batsman_Details'])


In [63]:
icc_tests_bbb_df.columns

Index(['Unnamed: 0', 'MatchID', 'Innings', 'Over', 'Id', 'Timestamp', 'Runs',
       'ZAD', 'Detail', 'Isball', 'Bowler', 'Bowler_Name', 'Batsman',
       'Batsman_Name', 'Commentary', 'Iswicket', 'Dismissed', 'Isboundary',
       'Summary', 'Batsman_Style', 'Non_Striker', 'Non_Striker_Name', 'Score',
       'Batsman_Runs', 'Bowler_Conceded_Runs', 'Dismissal_Type',
       'Dismissal_Id', 'Extras_Runs', 'Ball_Speed', 'Howout', 'Fielders',
       'Ball_Number', 'Day', 'Session', 'Milestone', 'Over_No', 'Ball',
       'Ball_Line_Length', 'This_Over', 'Is_Nonball_Wicket',
       'Edited_Commentary', 'End_Over', 'Default_Commentary', 'Review_Detail',
       'Run_Out_Miss', 'UID', 'OID', 'Bowler_Short_Name', 'Batsman_Short_Name',
       'Non_Striker_Short_Name', 'Ball_Speed_This_Over', 'Created_Timestamp',
       'IsFreehit', 'curr_non_striker_Runs', 'curr_non_striker_Balls',
       'curr_non_striker_Fours', 'curr_non_striker_Sixes',
       'curr_non_striker_Batsman', 'curr_bowler_Overs', 'c

In [64]:
icc_tests_bbb_df.drop(columns=['UID', 'OID', 'Bowler_Short_Name', 'Batsman_Short_Name', 'Non_Striker_Short_Name', 'This_Over', 'Ball_Speed_This_Over', 'Default_Commentary', 'Commentary', 'Id', 'Edited_Commentary', 'Isball'], inplace=True)
icc_tests_bbb_df.to_csv("./icc/icc_tests_bbb_final.csv", index=False)

In [65]:
icc_tests_bbb_df.columns

Index(['Unnamed: 0', 'MatchID', 'Innings', 'Over', 'Timestamp', 'Runs', 'ZAD',
       'Detail', 'Bowler', 'Bowler_Name', 'Batsman', 'Batsman_Name',
       'Iswicket', 'Dismissed', 'Isboundary', 'Summary', 'Batsman_Style',
       'Non_Striker', 'Non_Striker_Name', 'Score', 'Batsman_Runs',
       'Bowler_Conceded_Runs', 'Dismissal_Type', 'Dismissal_Id', 'Extras_Runs',
       'Ball_Speed', 'Howout', 'Fielders', 'Ball_Number', 'Day', 'Session',
       'Milestone', 'Over_No', 'Ball', 'Ball_Line_Length', 'Is_Nonball_Wicket',
       'End_Over', 'Review_Detail', 'Run_Out_Miss', 'Created_Timestamp',
       'IsFreehit', 'curr_non_striker_Runs', 'curr_non_striker_Balls',
       'curr_non_striker_Fours', 'curr_non_striker_Sixes',
       'curr_non_striker_Batsman', 'curr_bowler_Overs', 'curr_bowler_Maidens',
       'curr_bowler_Runs', 'curr_bowler_Wickets', 'curr_bowler_Bowler',
       'curr_striker_Runs', 'curr_striker_Balls', 'curr_striker_Fours',
       'curr_striker_Sixes', 'curr_striker_Batsma

In [66]:
import pandas as pd
df = pd.read_csv("./icc/icc_tests_bbb_final.csv")
df.columns

/tmp/ipykernel_25545/2846077226.py:2: DtypeWarning: Columns (16,18,19,22,23,25,26,27,31,34,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./icc/icc_tests_bbb_final.csv")


Index(['Unnamed: 0', 'MatchID', 'Innings', 'Over', 'Timestamp', 'Runs', 'ZAD',
       'Detail', 'Bowler', 'Bowler_Name', 'Batsman', 'Batsman_Name',
       'Iswicket', 'Dismissed', 'Isboundary', 'Summary', 'Batsman_Style',
       'Non_Striker', 'Non_Striker_Name', 'Score', 'Batsman_Runs',
       'Bowler_Conceded_Runs', 'Dismissal_Type', 'Dismissal_Id', 'Extras_Runs',
       'Ball_Speed', 'Howout', 'Fielders', 'Ball_Number', 'Day', 'Session',
       'Milestone', 'Over_No', 'Ball', 'Ball_Line_Length', 'Is_Nonball_Wicket',
       'End_Over', 'Review_Detail', 'Run_Out_Miss', 'Created_Timestamp',
       'IsFreehit', 'curr_non_striker_Runs', 'curr_non_striker_Balls',
       'curr_non_striker_Fours', 'curr_non_striker_Sixes',
       'curr_non_striker_Batsman', 'curr_bowler_Overs', 'curr_bowler_Maidens',
       'curr_bowler_Runs', 'curr_bowler_Wickets', 'curr_bowler_Bowler',
       'curr_striker_Runs', 'curr_striker_Balls', 'curr_striker_Fours',
       'curr_striker_Sixes', 'curr_striker_Batsma

In [67]:
columns = ['Unnamed: 0', 'MatchID', 'Innings', 'Over', 'Timestamp', 
       'Bowler', 'Bowler_Name', 'Batsman',
       'Batsman_Name', 'Batsman_Style', 'Non_Striker', 'Non_Striker_Name',

       'curr_bowler_Overs', 'curr_bowler_Maidens', 'curr_bowler_Runs', 'curr_bowler_Wickets', 'curr_bowler_Bowler', 
       'curr_striker_Runs', 'curr_striker_Balls', 'curr_striker_Fours', 'curr_striker_Sixes', 'curr_striker_Batsman', 
       'curr_non_striker_Runs', 'curr_non_striker_Balls', 'curr_non_striker_Fours', 'curr_non_striker_Sixes', 'curr_non_striker_Batsman',

       'Score', 'ZAD', 'Ball_Speed','Ball_Number', 'Day', 'Session', 'End_Over', 'Over_No', 'Ball', 'Ball_Line_Length',

       'Runs', 'Batsman_Runs', 'Bowler_Conceded_Runs', 'Extras_Runs', 'Detail', 'Isboundary',
       'Iswicket', 'Dismissed', 'Dismissal_Type','Dismissal_Id', 'Howout', 'Fielders',

       'Is_Nonball_Wicket', 'Milestone', 'Review_Detail', 'Run_Out_Miss', 'IsFreehit',
       'Summary']

df = df[columns]

In [68]:
df.columns
df.to_csv("./icc/icc_tests_bbb_final.csv", index=False)